![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Working with Standard Models Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information (called metadata) about data and the technology that supports it.

In this hands-on lab you will get to use Egeria to load and view a standard glossary and data model.

Standard models are provided by specialist organizations, consortiums and other types of industry groups to capture a description of the data that is typically used in a specific scenario.  Often the model includes descriptive information as well as structural information documenting how the data is organized and linked together.

In this lab, we are going to work with the Cloud Information Model (CIM) see [cloudinformationmodel.org](http://cloudinformationmodel.org).  This model covers the following subject areas:

* Party – people, their roles and organizations.
* Product – product descriptions, structures and packaging.
* Sales Order – customer orders for goods and services.
* Payment Method – payment methods including cards, coupons and digital wallets.
* Payment – individual payments for goods and services.
* Shipment – shipment of goods and services to the customer to fulfil an order.

This means the CIM can provide common data structures for new services spanning customer and employee interaction around typical commercial activities such as buying and selling of goods and services.

The motivation behind the cloud information model is to support organizations that are transforming their digital services to run on a variety of cloud platforms and with their own data centres.  Often, they are dealing with systems built on many different generations of technology, with data distributed amongst them.  The CIM provides a common language to describe the different types of data.

This is extremely valuable to the open metadata and governance ecosystem with its focus on building an open and transparent view of data across an organization. 

This lab notebook enables you to explore the contents of the CIM and explains how it can help you when setting up your own open metadata and governance ecosystem.


## The scenario

<img src="https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/erin-overview.png" style="float:left">

The ODPi Egeria team use the personas and scenarios from the fictitious company called Coco Pharmaceuticals. (See [https://opengovernance.odpi.org/coco-pharmaceuticals/](https://opengovernance.odpi.org/coco-pharmaceuticals/) for more information).

In this lab, we are joining **Erin Overview**, their information architect, as she works through the changes needed to their sales and supply chain processes to support the shift from selling traditional generic medicines to personalized medicine.
Erin's userId is `erinoverview`.

In [ ]:
erinsUserId  = "erinoverview"

<img src="https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/harry-hopeful.png" style="float:right">

Erin is working with **Harry Hopeful** from sales on how the hospitals will order products, issue invoices and make payments.
He has been talking to a number of the hospitals who are customer's of Coco Pharmaceuticals.  In general the hospitals are supportive of the new approach.  They see that they will not need to keep so much inventory since orders are per patient and just when needed.  All parties agree, however, that they need to protect the privacy of the patients and medical staff involved and not accidentally leak personal details through the order fulfillment process.

They need a clear definition of the process and how vital documents such as orders and invoices are filled out and protected. Erin volunteers to create a draft of the process and the associated data model.  She wants to use application independent terms in her definitions since they all use different systems.  She decides to use the CIM since it covers their problem space.


## Setting up

Coco Pharmaceuticals makes widespread use of ODPi Egeria for tracking and managing their data and related assets.
Figure 1 below shows their metadata servers and the Open Metadata and Governance (OMAG) Server Platforms that are hosting them.  Each metadata server supports a department in the organization.  The servers are distributed across the platform to even out the workload.  Servers can be moved to a different platform if needed.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms-metadata-server.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The code below sets up the network addresses for the three platforms.  This varies depending on whether you are running them locally, through **docker-compose** or on **kubernetes**.

In [ ]:
%run ../common/environment-check.ipynb

Erin is using the governance metadata server called `cocoMDS2`.  This service is hosted on the Core OMAG Server Platform.

----

## Loading the model content into a metadata server

Standard models are released in many different formats, some following open standards and others using a proprietary standard, often defined by a particular modeling tool.  Egeria typically provides a parser to read the specific model format and then a builder to convert the content into an open metadata archive and then a writer to write out the contents to a file for distribution as shown in figure 2.


![Figure 2](../images/design-model-to-archive-load.png)
> **Figure 2:** Creating and loading an open metadata archive

This hands on lab uses a version of the CIM model that is shipped as an open metadata archive with the Egeria release.
This may not be the latest version so if you wish to use this model in your project we recommend that you download
the latest version to the model and create a new archive.

The command to load the archive into the `cocoMDS2` metadata repository is shown below followed by a command to retrieve the CIM glossary description from the metadata repository.  The `glossaryId` is the unique identifier
for the CIM glossary.  The retrieved glossary description includes the title of the glossary and a description which is printed out.

In [ ]:
archiveFileName = 'content-packs/CloudInformationModel.json'
loadArchiveURL = cocoMDS2PlatformURL +  '/open-metadata/admin-services/users/' +  adminUserId + '/servers/' + cocoMDS2Name + '/instance/open-metadata-archives/file'

print(" ")
response = requests.post(loadArchiveURL, archiveFileName)
print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

getGlossaryURL = cocoMDS2PlatformURL + '/servers/' + cocoMDS2Name + '/open-metadata/access-services/glossary-view/users/' +  erinsUserId + '/glossaries'

response = requests.get(getGlossaryURL)

if response.status_code == 200:
    result = response.json().get('result')
    if result is not None:
        if len(result) > 0:
            for x in range(len(result)):
                glossary = result[x]
                glossaryId = glossary.get('guid')
                glossaryUsage = glossary.get('usage')
                glossaryDisplayName = glossary.get('displayName')
                print ("GlossaryId: " + glossaryId + "; " + glossaryDisplayName + ": " + glossaryUsage)
                glossaryDescription = glossary.get('description')
                print (" -> " + glossaryDescription)
        else:
            print ("No glossaries loaded ")
            print ("Response was:")
            prettyResponse = json.dumps(response.json(), indent=4)
            print (prettyResponse)
            print (" ")
    else:
        print ("No glossaries returned ")
else:
    print ("Unable to call server")


----
The glossary itself is organized into a hierarchy of **glossary categories**.  Within the categories are **glossary terms**.  Each term describes a concept or phrase used in the model.

The code below locates the root of the category hierarchy and navigates down the category hierarchy printing out the category names to reveal the structure of the glossary.


In [ ]:

indent = "    "
arrow = "--> "

glossaryViewURLRoot = cocoMDS2PlatformURL + '/servers/' + cocoMDS2Name +  '/open-metadata/access-services/glossary-view/users/' +  erinsUserId
getCategoryURLRoot = glossaryViewURLRoot + '/categories/'

getGlossaryMoreInfoURLRoot = cocoMDS2PlatformURL + '/servers/' + cocoMDS2Name + '/open-metadata/common-services/asset-consumer/connected-asset/users/' +  erinsUserId + '/referenceables/'
getGlossaryMoreInfoURLTail = '/more-information?elementStart=0&maxElements=50'


def getNestedCategories(whitespace, categoryGUID):
    response = requests.get(getCategoryURLRoot + categoryGUID + '/subcategories?from=0&size=100')
    result = response.json().get('result')
    for x in range(len(result)):
        guid = result[x].get('guid')
        getCategory(whitespace, guid)

    
def getCategory(whitespace, categoryGUID):
    response = requests.get(getCategoryURLRoot + categoryGUID)
    result = response.json().get('result')
    print (whitespace + arrow + result[0].get('displayName'))
    indentedWhitespace = whitespace + indent
    getNestedCategories(indentedWhitespace, categoryGUID)

response = requests.get(getGlossaryMoreInfoURLRoot + glossaryId + getGlossaryMoreInfoURLTail)
printResponse(response)
moreDetailList = response.json().get('list')
if moreDetailList:
    topLevelCategoryId = moreDetailList[0].get('guid')
    getCategory("", topLevelCategoryId)
           

----
As you can see there are two main categories: one for **property groups** and one for **subject areas**.  The property groups describe collections of related properties that appear in the model.  These are then used in
concept beads (objects) in the model.  The subject areas organize the concept beads into releated areas.

The next piece of code enables you to look at the details of a category and list the terms within it.
It is set up to view the **Account** category.  However you can view a different category by changing the name
in the search string.  If there are multiple categories with the same name then they will be displayed
one after the other.


In [ ]:

searchForCategory = "Party"

getAllGlossaryCategories = glossaryViewURLRoot + '/glossaries/' + glossaryId + '/categories?from=0&size=500'
getAllCategoryTermsRoot  = glossaryViewURLRoot + '/categories/'
getAllCategoryTermsTail  = '/terms?from=0&size=500'

nameWidth=30
descriptionWidth=80

def displayTerm(displayName, description):
    descriptionLength = len(description)
    if descriptionLength <= descriptionWidth:
        print('{0: <{1}}'.format(displayName, nameWidth) + ": "  + description)
    else:
        textPointer = 0
        print('{0: <{1}}'.format(displayName, nameWidth) + ": "  + description[textPointer:textPointer + descriptionWidth])
        textPointer = textPointer + descriptionWidth
        while textPointer < descriptionLength:
            print('{0: <{1}}'.format(" ", nameWidth) + ": "  + description[textPointer:textPointer + descriptionWidth])
            textPointer = textPointer + descriptionWidth

    
def getTermsForCategory(guid, displayName):
    response = requests.get(getAllCategoryTermsRoot + guid + getAllCategoryTermsTail)
    result = response.json().get('result')
    for x in range(len(result)):
        if x == 0:
            print(" ")
            print("Terms for category: " + displayName)
            print("-------------------------------------------------------------------------------------------")
        guid = result[x].get('guid')
        termName = result[x].get('displayName')
        description = result[x].get('description')
        displayTerm(termName, description)
        
def findCategories(searchString):
    categoryCount = 0
    response = requests.get(getAllGlossaryCategories)
    result = response.json().get('result')
    for x in range(len(result)):
        guid = result[x].get('guid')
        displayName = result[x].get('displayName')
        if displayName == searchString:
            getTermsForCategory(guid, displayName)
            categoryCount=categoryCount+1
    if categoryCount == 0:
        print("no results")
        
print(" ")
findCategories(searchForCategory)     
print(" ")


----

The final view of the glossary part of the CIM is a search for terms with similar names. You can change the setting of `searchName` to explore other terms.

In [ ]:

findGlossaryTermURL = cocoMDS2PlatformURL + '/servers/' + cocoMDS2Name +  '/open-metadata/access-services/asset-consumer/users/' +  erinsUserId + '/meanings/by-search-string?startFrom=0&pageSize=100'
getGlossaryTermURL = cocoMDS2PlatformURL + '/servers/' + cocoMDS2Name +  '/open-metadata/access-services/asset-consumer/users/' +  erinsUserId + '/meanings/by-name/?startFrom=0&pageSize=100'

def printMeanings(meanings):
    if meanings is not None:
        for x in range(len(meanings)):
            displayName = meanings[x].get('displayName')
            description = meanings[x].get('description')
            displayTerm(displayName, description)
    else:
        print("    no terms found")
    

def findTerms(searchString):
    response = requests.post(findGlossaryTermURL, searchString) 
    if response.status_code == 200:
        print("Results found for search string: " + searchString)
        print("-------------------------------------------------------------------------------------------")
        meanings = response.json().get('meanings')
        printMeanings(meanings)
    else:
        print("no results")


def getTerms(searchString):
    response = requests.post(findGlossaryTermURL, searchString)
    if response.status_code == 200:
        print("Terms exactly matching: " + searchString)
        print("-------------------------------------------------------------------------------------------")
        meanings = response.json().get('meanings')
        printMeanings(meanings)
    else:
        print("no results")

searchName="Party"
searchString=".*" + searchName + ".*"

print(" ")
findTerms(searchString)
print(" ")
print(" ")
getTerms(searchName)
print(" ")
